# 3rd Session - Finding alternative process performers

made by 임지택

- 어떠한 프로세스를 수행할 때 각각의 activity들에게 적절한 수행자가 assign 되어있습니다.
- 그러나 현직 수행자가 수행함에 있어서 문제가 발생하여 해당 activity를 수행 할 수 없는 상황이 발생 할 수 있습니다.
- 이럴 경우 회사에서는 그 activity를 가장 잘 수행 할 수 있는 사람을 골라 대체시켜야 합니다.
- 여러 alternative들 중에 best performer를 고르기 위해서는 3가지 관점을 고려해야 합니다.


    - Task execution
    - Transfer of work
    - Cycle time of an activity



- 이 3가지 관점에 각각 weight를 줘서 최종적으로 score가 가장 높은 사람이 최적의 alternative performer를 찾습니다.


![excel_file](excel_file.png)

In [1]:
import pandas as pd
from collections import Counter
import tkinter as tk
import networkx as nx   
from datetime import datetime
import duration


- tkinter: 파이썬에서 제공하는 gui 라이브러리, ui를 만들 시간이 많지 않아서 그냥 이걸로 썼습니다.
- duration: 시각을 표현하는데 있어서 초로 바꾸기 위하여 썼습니다.

In [4]:
def importfiles(f,rows,columns,activity,exe_weight,trans_weight,time_weight):
    f1 = open(f,'r')
    data =[]
    
    for line in f1:
        split_line=line.split(',')
        data.append(split_line)
    
    f1.close()
    headers = data[0]
    
    data = data[1:len(data)]
    apending = pd.DataFrame(data,columns =headers)
          
    activity_user = apending.groupby('Activity')['Resource'].apply(list)
    
    column = [i for i in range(len(columns))]

- 먼저 importfiles라는 함수를 만들었는데 import기능만 넣을려고 했는데 막코딩을 하다보니 저 함수 안에 모든 코드를 넣어버렸습니다...(코드가 많이 지저분하다는 점 죄송합니다)

- parameters로 무려 7가지를 받는데 다음과 같습니다 

(솔직히 지금 보니깐 참으로 헛짓거리를 많이 했다는 생각이 드네요...또 한번 죄송합니다.)


    - 파일 이름
    - row 이름
    - column 이름
    - acitivty 이름
    - task execution의 weight
    - transfer of work의 weight
    - cycle time의 weight

activity_user![](activity_user.png)


In [5]:
    matrix_activity = pd.DataFrame(index = rows, columns = column)

    a = list()
    for i in range(len(activity_user)):
        a.append([[x,(activity_user.values[i]).count(x)] for x in set(activity_user.values[i])])

    for k in range(len(a)):
        for i in range(len(matrix_activity)):
            for j in range(len(a[k])):
                if matrix_activity.index[i] == a[k][j][0]:
                    matrix_activity.loc[a[k][j][0],k] =a[k][j][1]

    matrix_activity = matrix_activity.fillna(0)
    matrix_activity.columns = columns


    matrix_activity = matrix_activity[activity]
    matrix_activity = matrix_activity[matrix_activity>0]
        

NameError: name 'rows' is not defined

- task execution에 관한 matrix_activity matrix를 만듭니다.

matrix_activity
![center](matrix_activity.png)

In [ ]:
    matrix_handover = pd.DataFrame(index = rows, columns = rows)
    
    activities = apending[apending['Activity'] == activity]
    pendingindex = activities.index
    pendingindex = list(pendingindex)
    
    def getbeforeafter(a):
        newindex=list()
        for i in range(len(a)):
            x=0
            y=0
            if a[i] == 0:
                return
            else:
                x = a[i]-1
                y = a[i]+1
            newindex.append(x)
            newindex.append(a[i])
            newindex.append(y)

        return newindex
    
    newindex = getbeforeafter(pendingindex)

In [ ]:
    new = list()
    for i in range(len(newindex)):
        new.append(apending[apending.index == newindex[i]])
    
    new_pd = pd.DataFrame(new[0])
    
    for i in range(1,len(new)):
        new_pd = new_pd.append(new[i])
           
    handover_user = apending.groupby('Case ID')['Resource'].apply(list)
    handover_user = list(handover_user)
    
    
    hand = list()
    for i in range(len(handover_user)):
        for j in range(len(handover_user[i])-1):
            hand.append((handover_user[i][j],handover_user[i][j+1]))    
    
    aca = Counter(hand)
    
    usertouser = aca.keys()
    usertouser = list(usertouser)
    
    usertofreq = aca.values()
    usertofreq = list(usertofreq)
    
    for i in range(len(matrix_handover)):
        for j in range(len(matrix_handover)):    
            for k in range(len(usertouser)):
                if (matrix_handover.index[i] == usertouser[k][0]) and (matrix_handover.columns[j] == usertouser[k][1]):
                    matrix_handover.loc[usertouser[k][0],usertouser[k][1]] = usertofreq[k]
                    
                    
    matrix_handover = matrix_handover.fillna(0)

- Transfer of work에 관한 matrix_handover matrix를 만듭니다.

matrix_handover

![center](matrix_handover.png)

In [ ]:
    timediff = list()
    
    activities = activities.reset_index()
    
    for i in range(len(activities)):
        time =datetime.strptime(activities['Start Timestamp'][i],'%Y-%m-%d %H:%M') -datetime.strptime(activities['Complete Timestamp'][i],'%Y-%m-%d %H:%M')
        timediff.append(time)
    
    timediff = list(timediff)
    
    timeinsecond= list()
    for i in range(len(timediff)):
        timeinsecond.append(duration.to_seconds(timediff[i]))
    
    timedf = pd.DataFrame(index= activities['Resource'])
    timedf['timediff'] = timeinsecond
    
    timedf = timedf.groupby(timedf.index)['timediff'].sum()
    timedf = timedf[timedf >0]      
    
    time_total = pd.DataFrame(index = users_index, columns = users_index)
    
    
    for i in range(len(time_total)):
        for j in range(len(time_total)):
            if (time_total.index[i] in timedf.index) and (time_total.columns[j] in timedf.index):
                if (timedf.loc[time_total.index[i]] != 0) and (timedf.loc[time_total.index[j]] != 0): #¼ýÀÚ/¼ýÀÚ                 
                    time_total.loc[time_total.index[i],time_total.columns[j]] = timedf.loc[time_total.columns[j]] / timedf.loc[time_total.index[i]]                                                   
            else:
                time_total.loc[time_total.index[i],time_total.columns[j]] = 0
    
    for i in range(len(time_total)):
        for j in range(len(time_total)):
            if time_total.loc[time_total.index[i]][j] !=0:
                time_total.loc[time_total.index[i]][j] = 1.0/time_total.loc[time_total.index[i]][j]

- Cycle time of specific activity에 관한 time_total matrix를 만듭니다.

time_total
![center](DOS_time.png)

In [ ]:
    def doscalculator(a,b):
        x=0.0
        y=0.0
        for i in range(len(a)):
            if a[i] > b[i]:
                x+= b[i]/a[i]
            elif a[i] < b[i]: 
                x+=1
            if a[i] > 0:
                y+=1
            else:
                y+=0
                
        return x/y   
    
    DOS_task = pd.DataFrame(index = rows, columns = rows)
    
    for i in range(len(DOS_task)):
        for j in range(len(DOS_task)):
            if i ==j:
                DOS_task.loc[DOS_task.index[i],DOS_task.columns[j]] = 1
            if (DOS_task.index[i] in matrix_activity.index) and (DOS_task.columns[j] in matrix_activity.index):
                DOS_task.loc[DOS_task.index[i],DOS_task.columns[j]] = matrix_activity.loc[DOS_task.columns[j]] / matrix_activity.loc[DOS_task.index[i]]
            else:
                DOS_task.loc[DOS_task.index[i],DOS_task.columns[j]] =0        

        
    DOS_transfer = pd.DataFrame(index = rows, columns = rows)
    
    for i in range(len(DOS_transfer)):
        for j in range(len(DOS_transfer)):
            DOS_transfer.loc[DOS_transfer.index[i],DOS_transfer.columns[j]] = doscalculator(list(matrix_handover.loc[DOS_transfer.index[i],]),list(matrix_handover.loc[DOS_transfer.columns[j]]))
            if i==j:
                DOS_transfer.loc[DOS_transfer.index[i],DOS_transfer.columns[j]] = 1

- 여기서 DOS라는 개념을 저희 연구실 형한테 아이디어를 빌렸습니다.
- DOS는 Degree Of Substitution의 약자인데 두 명의 performer가 있고 그들의 profile이 각각 존재합니다.
    - A = {a1,a2,a3, ... ,an}
    - B = {b1,b2,b3, ..., bn}
    
- B가 A를 대체할 DOS값은 다음과 같이 정의됩니다.

![](DOS.png)

- 앞에서 구한 두가지 matrix 즉, matrix_activity 와 matrix_handover에 대해 DOS 연산을 실행해준 뒤 DOS_task와 DOS_transfer dataframe에 넣어줍니다.




DOS_task
![](DOS_task.png)



DOS_transfer
![](DOS_transfer.png)

In [ ]:
    Exe_weight = exe_weight
    Trans_weight = trans_weight
    Time_weight = time_weight
    
    DOS_Time = pd.DataFrame(index = users_index, columns = users_index)
    
    for i in range(len(DOS_Time)):
        for j in range(len(DOS_Time)):
            DOS_Time.loc[DOS_Time.index[i]][j]  = DOS_task.loc[DOS_task.index[i]][j]*Exe_weight + DOS_transfer.loc[DOS_transfer.index[i]][j]*Trans_weight + time_total.loc[time_total.index[i]][j]*Time_weight
        
    realmax = list()            
    for i in range(len(DOS_Time)):
        values = set(list(DOS_Time.loc[DOS_Time.index[i]]))
        values.discard(DOS_Time.loc[DOS_Time.index[i]][i])
        realmax.append(max(values))            
    
    indices = list()
    for i in range(len(realmax)):
        indices.append(list(DOS_Time.loc[DOS_Time.index[i]]).index(realmax[i]))   
        
    bestresourceindex = [str(i+1) for i in indices]
    
        
    Bestresource = pd.DataFrame(index = users_index)
        
    Bestresource['Replace resource'] = bestresourceindex
    Bestresource['Replace score'] = realmax

- 3개의 matrix에 대해서 weight를 부여하여 최종적인 dataframe인 Bestresource를 만듭니다.

In [ ]:
    class SampleApp(tk.Tk):
        def __init__(self, *args, **kwargs):
            tk.Tk.__init__(self, *args, **kwargs)
            lb = tk.Listbox(self)
            for i in range(len(DOS_Time.index)):
                lb.insert("end",DOS_Time.index[i])
            lb.bind("<Double-Button-1>", self.OnDouble)
            lb.pack(side="top", fill="both", expand=True)
    
        def OnDouble(self, event):
            widget = event.widget
            selection=widget.curselection()
            value = widget.get(selection[0])
            print("selection:", selection, ": '%s'" % value)
            g = nx.Graph()
            for i in range(len(DOS_Time.columns)):
                g.add_edge(value,DOS_Time.columns[i], weight = DOS_Time[value][i])
            nx.draw_networkx(g)
            plt.show()
            print(DOS_Time.loc[value])
            print("Best resource is" ,Bestresource.loc[value][0], "and its score is" ,Bestresource.loc[value][1])
    
    if __name__ == "__main__":
        app = SampleApp()
        app.mainloop()
        
    return Bestresource

- 원래 계획에는 없었지만 교수님의 제안으로 ui를 만들었습니다.
- 몇일 밖에 남지 않아서 tkinter를 사용하였고 코드도 구글링으로 가져온거라 왜이렇게 짰다고 물어보신다면 잘 모릅니다.
- 먼저 listbox를 만들고 대체할 performer를 선택하면 거기에 해당되는 최적의 alternative performer가 출력이 되고 나머지 사람들에 대한 score값도 제공이 됩니다.

![center](listbox.png)
![center](result_2.1.png)
![center](result_3.png)

In [ ]:
columnss = ['A_Accepted', 'A_Complete', 'A_Concept', 'A_Create Application',
       'A_Incomplete', 'A_Pending', 'A_Submitted', 'A_Validating',
       'O_Accepted', 'O_Cancelled', 'O_Create Offer', 'O_Created',
       'O_Returned', 'O_Sent (mail and online)', 'O_Sent (online only)',
       'W_Assess potential fraud', 'W_Call after offers',
       'W_Call incomplete files', 'W_Complete application', 'W_Handle leads',
       'W_Shortened completion', 'W_Validate application']
users_index = ['User_'+str(i) for i in range(1,144)]

activity = 'W_Validate application'
exe_weight = 0.3
trans_weight = 0.1
time_weight =0.6

importfiles('D:\\Apending_time.csv',users_index,columnss,activity,exe_weight,trans_weight, time_weight)

